# Задание 1. Регрессия


**Используя матричный метод напишите через запятую оценки коэффициентов линейной регрессии D на V используя данные о длине тормозного пути и скорости для трёх автомобилейю D = [10, 7, 12], V = [60, 50, 75].**

In [0]:
import numpy as np
from np import insert
from numpy import linalg

# Напишите соответствующий код

#Задание 2. Персептрон

**Здесь вам предстоит дописать функции класса Персептрон и с его помощью решить задачу классификации. По данным о цвете и семметричности нужно предсказать, что это за фрукт (яблоко или груша). **

In [0]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d as p3
import random
import time
from functools import partial
from numpy.linalg import norm
from google.colab import files
import pandas as pd
import io

%matplotlib inline
random.seed(42)

In [0]:
uploaded = files.upload()
data = pd.read_csv(io.StringIO(uploaded['data.csv'].decode('utf-8')), header = None)
data = np.array(data)
data

In [0]:
pears = data[:, 2] == 1
apples = np.logical_not(pears)
fig, ax = plt.subplots(figsize=(10, 7), facecolor = 'w')
plt.scatter(data[apples][:, 0], data[apples][:, 1], color = "red")
plt.scatter(data[pears][:, 0], data[pears][:, 1], color = "green")
plt.xlabel("yellowness")
plt.ylabel("symmetry")
plt.show()

In [0]:
class Perceptron:

    def __init__(self, w, b):
        """
        Инициализируем наш объект - перцептрон.
        w - вектор весов размера (m, 1), где m - количество переменных
        b - число
        """
        
        self.w = w
        self.b = b

    def forward_pass(self, single_input):
        """
        Метод рассчитывает ответ перцептрона при предъявлении одного примера
        single_input - вектор примера размера (m, 1).
        Метод возвращает число (0 или 1) или boolean (True/False)
        """
        
        result = 0
        for i in range(0, len(self.w)):
            result += self.w[i] * single_input[i]
        result += self.b
        
        if result > 0:
            return 1
        else:
            return 0

    def vectorized_forward_pass(self, input_matrix):
        """
        Метод рассчитывает ответ перцептрона при предъявлении набора примеров
        input_matrix - матрица примеров размера (n, m), каждая строка - отдельный пример,
        n - количество примеров, m - количество переменных
        Возвращает вертикальный вектор размера (n, 1) с ответами перцептрона
        (элементы вектора - boolean или целые числа (0 или 1))
        """

        # Этот метод необходимо реализовать
        
        y = np.array(self.w.T@input_matrix.T+self.b > 0)
        return y
    
    def train_on_single_example(self, example, y):
        """
        Принимает вектор активации входов example формы (m, 1) 
        и правильный ответ для него (число 0 или 1 или boolean),
        обновляет значения весов перцептрона в соответствии с этим примером
        и возвращает размер ошибки, которая случилась на этом примере до изменения весов (0 или 1)
        (на её основании мы потом построим интересный график)
        """

        # Этот метод необходимо реализовать
        
        y1 = self.w.T@example + self.b > 0 
        delta = (y-y1)
        self.w += delta*example
        self.b += delta
        return delta
        
        
    def train(self, input_matrix, y, max_steps=1e8):
        """
        input_matrix - матрица входов размера (n, m),
        y - вектор правильных ответов размера (n, 1) (y[i] - правильный ответ на пример input_matrix[i]),
        max_steps - максимальное количество шагов.
        """
        
        i = 0
        errors = 1
        while errors and i < max_steps:
            i += 1
            errors = 0
            for example, answer in zip(input_matrix, y):
                example = example.reshape((example.size, 1))
                error = self.train_on_single_example(example, answer)
                errors += int(error)
                
              
    # По желанию, можете реализовать метод train в матричном виде

In [0]:
def create_perceptron(m):
    """Создаём перцептрон со случайными весами и m входами"""
    
    w = np.random.random((m, 1))
    return Perceptron(w, 1)

In [0]:
def test_v_f_p(n, m):
    """
    Расчитывает для перцептрона с m входами
    с помощью методов forward_pass и vectorized_forward_pass
    n ответов перцептрона на случайных данных.
    Возвращает время, затраченное vectorized_forward_pass и forward_pass
    на эти расчёты.
    """
    
    p = create_perceptron(m)
    input_m = np.random.random_sample((n, m))
    
    start = time.clock()
    vec = p.vectorized_forward_pass(input_m)
    end = time.clock()
    vector_time = end - start
    
    start = time.clock()
    for i in range(0, n):
        p.forward_pass(input_m[i]) 
    end = time.clock()
    plain_time = end - start

    return [vector_time, plain_time]
  
  
def mean_execution_time(n, m, trials=100):
    """среднее время выполнения forward_pass и vectorized_forward_pass за trials испытаний"""
    
    return np.array([test_v_f_p(m, n) for _ in range(trials)]).mean(axis=0)

def plot_mean_execution_time(n, m):
    """рисует графики среднего времени выполнения forward_pass и vectorized_forward_pass"""
    
    mean_vectorized, mean_plain = mean_execution_time(int(n), int(m))
    p1 = plt.bar([0], mean_vectorized,  color='g')
    p2 = plt.bar([1], mean_plain, color='r')
    plt.ylabel("Time spent")
    plt.yticks([])
    plt.xticks([])
    plt.legend(("vectorized","non - vectorized"))
    plt.show()

    
plot_mean_execution_time(random.randint(10, 50), random.randint(10, 50))

In [0]:
def accuracy(y, y1):
    return np.mean((y-y1)**2)
  

from sklearn.model_selection import train_test_split
"""Создает обучающую и тестовую выборку, обучите персептрон и получите ошибку предсказания"""

# Напишите соответствующий код  
 
from sklearn.model_selection import train_test_split
X = data[:, 0:2]
X = np.insert(X, 0, 1, axis = 1)
y = data[:, 2]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
p = create_perceptron(3)
p.train_until_convergence(x_train, y_train)
print (accuracy(y_test, p.vectorized_forward_pass(x_test)))